In [1]:
from langchain.document_loaders import WebBaseLoader

# List of URLs for stock data
urls = [
    "https://finance.yahoo.com/quote/AAPL",  # Yahoo Finance - Apple
    "https://www.investing.com/equities/apple-computer-inc",  # Investing.com - Apple
    "https://www.moneycontrol.com/india/stockpricequote/technology/infosys/IT"  # Moneycontrol - Infosys
]

# Load documents from all URLs
loader = WebBaseLoader(urls)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")


USER_AGENT environment variable not set, consider setting it to identify your requests.


Number of documents loaded: 3


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter()

# Split documents into chunks
doc_chunks = text_splitter.split_documents(docs)
print(f"Number of document chunks: {len(doc_chunks)}")


Number of document chunks: 31


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceEmbeddings(
    api_key=HF_token, model_name="BAAI/bge-base-en-v1.5"
)